# Distributing Jupyter Extensions as Python Packages

## How can the notebook be extended?
The Jupyter Notebook server and client application are both deeply customizable. Their behavior can be extended by, respectively:

- nbextension:
    - a single JS file, or directory of JavaScript, Cascading StyleSheets, etc. that contain at least
        - a JavaScript module packaged as an [AMD modules](https://en.wikipedia.org/wiki/Asynchronous_module_definition)
            - that exports a function `load_ipython_extension`
- server extension: an importable Python module
    - that implements `load_jupyter_server_extension`

Since it is somewhat rare to have a server extension that does not have any frontend components, for convenience ad consistency, all these assets can be versioned together, with a few simple commands to make installing them easy and less error-prone for the user. 

## Install a Python Package with Jupyter Extensions
There are many ways someone may get the enclosing python package:
```shell
pip install some_package
# or
conda install some_package
# or
apt-get install some_package
```

## Enable a Server Extension

For the simplest case, a server extension with no frontend components, a `pip` user that wants their configuration stored in their home directory would type:
```shell
jupyter serverextension enable --py some_package --user
```

A `virtualenv` or `conda` user can pass `--sys-prefix` insted of `--user` to keep their environment reproducible.
```shell
[source] activate my-environment
jupyter serverextension enable --py some_package --sys-prefix
```

## Install the nbextension assets

In any event, if a package also has frontend assets that must be available (but not neccessarily enabled by default):
```shell
jupyter nbextension install --py some_package --user # or --sys-prefix
```

## Enable nbextension assets

Finally, if a package has extensions that need to be required in the browser without user interactions,
```shell
jupyter nbextension enable --py some_package --user # or --sys-prefix
```

## Did it work?
After running some extension install steps, one can list what is presently known about, whether it's enabled, etc.

```shell
jupyter nbextension list
jupyter serverextension list
```

> Of course, in addition to the files listed, there are number of other files one needs to build a proper package. Here are some good resources:
- [The Hitchhiker's Guide to Packaging](http://the-hitchhikers-guide-to-packaging.readthedocs.org/en/latest/quickstart.html)
- [Repository Structure and Python](http://www.kennethreitz.org/essays/repository-structure-and-python) by Kenneth Reitz

> How you distribute them, too, is important:
- [Packaging and Distributing Projects](http://python-packaging-user-guide.readthedocs.org/en/latest/distributing/)
- [conda: Building packages](http://conda.pydata.org/docs/building/build.html)

> Here are some tools to get you started:
- [generator-nbextension](https://github.com/Anaconda-Server/generator-nbextension)

# Implementing Python-wrapped extensions

Here is an example of a python module which contains a server extension directly on itself. It has this directory structure
```
- setup.py
- MANIFEST.in
- my_module/
  - __init__.py
```

### `my_module/__init__.py`

```python
def _jupyter_server_extension_paths():
    return [{
        "module": "my_module"
    }]


def load_jupyter_server_extension(nbapp):
    nbapp.log.info("my module enabled!")
```

Which a user can install with:
```
jupyter serverextension enable --py my_module [--sys-prefix|--user]
```

Here is another server extension, with a front-end module.  It assumes this directory structure:

```
- setup.py
- MANIFEST.in
- my_fancy_module/
  - __init__.py
  - static/
    index.js
```

### `my_fancy_module/__init__.py`

```python
def _jupyter_server_extension_paths():
    return [{
        "module": "my_fancy_module"
    }]

# Jupyter Extension points
def _jupyter_nbextension_paths():
    return [dict(
        section="notebook",
        # the path is relative to the `my_fancy_module` directory
        src="static",
        # directory in the `nbextension/` namespace
        dest="my_fancy_module",
        # _also_ in the `nbextension/` namespace
        require="my_fancy_module/index")]

def load_jupyter_server_extension(nbapp):
    nbapp.log.info("my module enabled!")
```

Which a user can install with:
```
jupyter serverextension install --py my_fancy_module [--sys-prefix|--user]
jupyter nbextension install --py my_fancy_module [--sys-prefix|--user]
jupyter nbextension enable --py my_fancy_module [--sys-prefix|--user]
```